<a href="https://colab.research.google.com/github/halnegheimish/ForcedInvalidation/blob/main/notebooks/MNLI_FI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code in this notebook is based on https://github.com/huggingface/notebooks/blob/6ca682955173cc9d36ffa431ddda505a048cbe80/examples/text_classification.ipynb

# preliminaries

In [ ]:
! pip install datasets transformers

In [ ]:
!apt install subversion

In [ ]:
!mkdir data

In [29]:
cd data

/content/data


In [ ]:
!svn checkout https://github.com/halnegheimish/ForcedInvalidation/trunk/data/eval/mnli/
!svn checkout https://github.com/halnegheimish/ForcedInvalidation/trunk/data/eval/anli/


In [ ]:
!gdown --fuzzy --folder https://drive.google.com/drive/folders/1UuqBfHn2sOu7KyCajegtuE6gfXLCsvIc?usp=share_link

In [30]:
cd ..

/content


In [ ]:
import itertools
import random
import numpy as np
import torch
import pandas as pd

seed= 42
random.seed(seed)
np.random.seed(seed)

from transformers import set_seed

set_seed(seed)


In [ ]:
from datasets import load_dataset, load_metric

metric = load_metric('glue', 'mnli')

#train model

In [ ]:
from datasets import load_from_disk

In [ ]:
ngrams_train = load_from_disk('data/mnli/mnli_train_ngrams')

In [ ]:
ngrams_dev= load_from_disk('data/mnli/mnli_val_ngrams')

In [ ]:
model_checkpoint = "bert-base-uncased"
batch_size = 16

In [ ]:
task='mnli'

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [ ]:
encoded_train = ngrams_train.map(preprocess_function, batched=True)

In [ ]:
encoded_val = ngrams_dev.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 4 
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    output_dir=f'bert_base_finetuned_mnli_ngrams_{seed}'
)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return  metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

#evaluation

In [ ]:
datasets=[dict(path='data/mnli/mnli_mm_dev_1gram_p' , name='mnli_mm_1g_p', split='mismatched'),
          dict(path='data/mnli/mnli_mm_dev_2gram_p' , name='mnli_mm_2g_p', split='mismatched'),
          dict(path='data/mnli/mnli_mm_dev_3gram_p' , name='mnli_mm_3g_p', split='mismatched'),
          dict(path='data/mnli/mnli_mm_dev_1gram_h' , name='mnli_mm_1g_h', split='mismatched'),
          dict(path='data/mnli/mnli_mm_dev_2gram_h' , name='mnli_mm_2g_h', split='mismatched'),
          dict(path='data/mnli/mnli_mm_dev_3gram_h' , name='mnli_mm_3g_h', split='mismatched'),
          dict(path='data/mnli/mnli_dev_1gram_p' , name='mnli_1g_p', split='matched'),
          dict(path='data/mnli/mnli_dev_2gram_p' , name='mnli_2g_p', split='matched'),
          dict(path='data/mnli/mnli_dev_3gram_p' , name='mnli_3g_p', split='matched'),
          dict(path='data/mnli/mnli_dev_1gram_h' , name='mnli_1g_h', split='matched'),
          dict(path='data/mnli/mnli_dev_2gram_h' , name='mnli_2g_h', split='matched'),
          dict(path='data/mnli/mnli_dev_3gram_h' , name='mnli_3g_h', split='matched'),
          dict(path='data/anli/anli_dev_r1_1gram_p' , name='anli_r1_1g_p', split='anli_r1'),
          dict(path='data/anli/anli_dev_r1_2gram_p' , name='anli_r1_2g_p', split='anli_r1'),
          dict(path='data/anli/anli_dev_r1_3gram_p' , name='anli_r1_3g_p', split='anli_r1'),
          dict(path='data/anli/anli_dev_r1_1gram_h' , name='anli_r1_1g_h', split='anli_r1'),
          dict(path='data/anli/anli_dev_r1_2gram_h' , name='anli_r1_2g_h', split='anli_r1'),
          dict(path='data/anli/anli_dev_r1_3gram_h' , name='anli_r1_3g_h', split='anli_r1'),
          dict(path='data/anli/anli_dev_r2_1gram_p' , name='anli_r2_1g_p', split='anli_r2'),
          dict(path='data/anli/anli_dev_r2_2gram_p' , name='anli_r2_2g_p', split='anli_r2'),
          dict(path='data/anli/anli_dev_r2_3gram_p' , name='anli_r2_3g_p', split='anli_r2'),
          dict(path='data/anli/anli_dev_r2_1gram_h' , name='anli_r2_1g_h', split='anli_r2'),
          dict(path='data/anli/anli_dev_r2_2gram_h' , name='anli_r2_2g_h', split='anli_r2'),
          dict(path='data/anli/anli_dev_r2_3gram_h' , name='anli_r2_3g_h', split='anli_r2'),
          dict(path='data/anli/anli_dev_r3_1gram_p' , name='anli_r3_1g_p', split='anli_r3'),
          dict(path='data/anli/anli_dev_r3_2gram_p' , name='anli_r3_2g_p', split='anli_r3'),
          dict(path='data/anli/anli_dev_r3_3gram_p' , name='anli_r3_3g_p', split='anli_r3'),
          dict(path='data/anli/anli_dev_r3_1gram_h' , name='anli_r3_1g_h', split='anli_r3'),
          dict(path='data/anli/anli_dev_r3_2gram_h' , name='anli_r3_2g_h', split='anli_r3'),
          dict(path='data/anli/anli_dev_r3_3gram_h' , name='anli_r3_3g_h', split='anli_r3'), 
    ]

runs=[dict(path='bert_base_finetuned_mnli_ngrams_42/checkpoint-211795/',
           name='invalid', seed=42),
      ]

In [ ]:
def load_model(path):
  tokenizer = AutoTokenizer.from_pretrained(path)
  model = AutoModelForSequenceClassification.from_pretrained(path)
  model_checkpoint=path
  model_name = model_checkpoint.split("/")[-2]
  args = TrainingArguments(
    f"{model_name}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    do_train= False,
    do_eval=  True,
    per_device_eval_batch_size=16,
  )
  return tokenizer, model, args


def eval_dataset(dataset, tokenizer, model, args):
  encoded_dataset= dataset.map(preprocess_function, batched=True)
  trainer = Trainer(
    model,
    args,
    train_dataset=None,
    eval_dataset=encoded_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
  ) 
  metrics= trainer.evaluate()
  return metrics

def eval_dataset(dataset, tokenizer, model, args):
  encoded_dataset= dataset.map(preprocess_function, batched=True)
  trainer = Trainer(
    model,
    args,
    train_dataset=None,
    eval_dataset=encoded_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
  ) 
  metrics= trainer.evaluate()
  return metrics

def eval_mnli(tokenizer, model, args):
  dataset= load_dataset('glue', 'mnli')['validation_matched']
  encoded_dataset= dataset.map(preprocess_function, batched=True)
  trainer = Trainer(
    model,
    args,
    train_dataset=None,
    eval_dataset=encoded_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
  ) 
  metrics= trainer.evaluate()
  return metrics

def eval_mnli_mm(tokenizer, model, args):
  dataset= load_dataset('glue', 'mnli')['validation_mismatched']
  encoded_dataset= dataset.map(preprocess_function, batched=True)
  trainer = Trainer(
    model,
    args,
    train_dataset=None,
    eval_dataset=encoded_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
  ) 
  metrics= trainer.evaluate()
  return metrics

def eval_anli(tokenizer, model, args, split='r3'):
  dataset= load_dataset('anli')[f'dev_{split}']
  encoded_dataset= dataset.map(preprocess_function, batched=True)
  trainer = Trainer(
    model,
    args,
    train_dataset=None,
    eval_dataset=encoded_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
  ) 
  metrics= trainer.evaluate()
  return metrics

In [ ]:
results_all={}
for run in runs:
  tokenizer, model, args= load_model(run['path'])
  results_all[f'seed_{run["seed"]}']={}
  results_all[f'seed_{run["seed"]}']['dev_matched']=eval_mnli(tokenizer, model, args)['eval_accuracy']
  results_all[f'seed_{run["seed"]}']['dev_mismatched']=eval_mnli_mm(tokenizer, model, args)['eval_accuracy']
  results_all[f'seed_{run["seed"]}']['dev_anli_r1']=eval_anli(tokenizer, model, args, split='r1')['eval_accuracy']
  results_all[f'seed_{run["seed"]}']['dev_anli_r2']=eval_anli(tokenizer, model, args, split='r2')['eval_accuracy']
  results_all[f'seed_{run["seed"]}']['dev_anli_r3']=eval_anli(tokenizer, model, args, split='r3')['eval_accuracy']
  for dp in datasets:
    dataset= load_from_disk(dp['path'])
    acc= eval_dataset(dataset, tokenizer, model, args)['eval_accuracy']
    results_all[f'seed_{run["seed"]}'][dp['name']]=acc




In [ ]:
df=pd.DataFrame(results_all)

In [ ]:
df

,seed_42
anli_r1_1g_h,0.001001
anli_r1_1g_p,0.000000
anli_r1_2g_h,0.006006
anli_r1_2g_p,0.000000
anli_r1_3g_h,0.015015
anli_r1_3g_p,0.001001
anli_r2_1g_h,0.002002
anli_r2_1g_p,0.000000
anli_r2_2g_h,0.010010
anli_r2_2g_p,0.000000
